Esta notebook contiene bloques de código útiles para realizar Q-learning en el entorno "Pendulum"

In [1]:
import numpy as np
from pendulum_env_extended import PendulumEnvExtended
import random 

In [2]:
env = PendulumEnvExtended(render_mode='rgb_array')

Discretización de los estados

In [3]:
x_space = np.linspace(-1, 1, 10)
y_space = np.linspace(-1, 1, 10)
vel_space = np.linspace(-8, 8, 50)
x_space

array([-1.        , -0.77777778, -0.55555556, -0.33333333, -0.11111111,
        0.11111111,  0.33333333,  0.55555556,  0.77777778,  1.        ])

Obtener el estado a partir de la observación

In [4]:
def get_state(obs):
    x, y, vel = obs
    x_bin = np.digitize(x, x_space)
    y_bin = np.digitize(y, y_space)
    vel_bin = np.digitize(vel, vel_space)
    return x_bin, y_bin, vel_bin

In [5]:
state = get_state(np.array([-0.4, 0.2, 0.3]))
state

(3, 6, 26)

Discretización de las acciones

In [6]:
actions = list(np.linspace(-2, 2, 10))
actions

[-2.0,
 -1.5555555555555556,
 -1.1111111111111112,
 -0.6666666666666667,
 -0.22222222222222232,
 0.22222222222222232,
 0.6666666666666665,
 1.1111111111111107,
 1.5555555555555554,
 2.0]

In [7]:
def get_sample_action():
    return random.choice(actions)

Inicilización de la tabla Q

In [8]:
Q = np.zeros((len(x_space) + 1, len(y_space) + 1, len(vel_space) + 1, len(actions)))
Q

array([[[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
    

Obtención de la acción a partir de la tabla Q

In [9]:
def optimal_policy(state, Q):
    action = actions[np.argmax(Q[state])]
    return action

Epsilon-Greedy Policy

In [10]:
def epsilon_greedy_policy(state, Q, epsilon=0.1):
    explore = np.random.binomial(1, epsilon)
    if explore:
        action = get_sample_action()
        print('explore')
    # exploit
    else:
        action = optimal_policy(state, Q)
        print('exploit')
        
    return action

In [11]:
import wandb

wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: naistat-a (andynaistat). Use `wandb login --relogin` to force relogin


True

Ejemplo de episodio 

In [12]:
import numpy as np
import wandb

def train(env, Q, alpha=0.1, gamma=0.9, epsilon=0.1, epsilon_min=0.01, epsilon_decay=0.995, episodes=100):
    for i in range(episodes):
        obs,_ = env.reset()
        done = False
        total_reward = 0
        state = get_state(obs)
        
        while not done:
            # Acción del modelo
            action = epsilon_greedy_policy(state, Q, epsilon)
            
            # Indice de la accion en Q
            action_idx = actions.index(action)
            
            # Acción del ambiente
            real_action = np.array([action])
            
            obs, reward, done, _, _ = env.step(real_action)
            next_state = get_state(obs)
            
            # Usar action_idx para actualizar Q
            Q[state][action_idx] += alpha * (reward + gamma * np.max(Q[next_state]) - Q[state][action_idx])
            
            state = next_state
            total_reward += reward
        
        # Reducir epsilon
        epsilon = max(epsilon_min, epsilon * epsilon_decay)
        
        # Registrar recompensa total
        wandb.log({'episode': i, 'total_reward': total_reward, 'epsilon': epsilon, "avg_q_value": np.mean(Q), "reward": reward})

        #print('->', state, action, reward, obs, done)
    #print('total_reward', total_reward)

In [13]:
import matplotlib.pyplot as plt

def imgRender():
    img = env.render()
    plt.imshow(img)
    plt.axis('off')
    plt.show()

In [ ]:
# test the agent
obs,_ = env.reset()
done = False

# Configurar el gráfico
fig = plt.figure()
img = plt.imshow(env.render())

while not done:
    state = obs
    action = optimal_policy(get_state(obs), Q)
    obs, reward, done, _, _ = env.step(np.array([action]))
    
    img.set_data(env.render())
    display.display(plt.gcf())
    display.clear_output(wait=True)
    #print('->', state, action, reward, obs, done)

In [ ]:
# inicializar la corrida
wandb.init(
    # establecer el nombre del proyecto
    project="pendulum",

    # establecer hiperparámetros
    config={
    "alpha": 0.3,
    "gamma": 0.9,
    "epsilon": 0.5,
    "epochs": 1000,
    "epsilon_min":0.01, 
    "epsilon_decay":0.995
    }
)

# entrenar el agente
train(env, Q, wandb.config.alpha, wandb.config.gamma, wandb.config.epsilon, wandb.config.epsilon_min, wandb.config.epsilon_decay, wandb.config.epochs)

# finalizar la corrida
wandb.finish()

In [ ]:
import wandb

# Definir la configuración del sweep
sweep_config = {
    'method': 'grid',  # o 'random'
    'metric': {
        'name': 'reward',
        'goal': 'maximize'
    },
    'parameters': {
        'alpha': {
            'values': [0.1, 0.2, 0.3]
        },
        'gamma': {
            'values': [0.8, 0.9, 1.0]
        },
        'epsilon': {
            'values': [0.1, 0.3, 0.5]
        },
        'epochs': {
            'values': [1]
        },
        'epsilon_min': {
            'values': [0.01]
        },
        'epsilon_decay': {
            'values': [0.995]
        }
    }
}

# Crear el sweep
sweep_id = wandb.sweep(sweep_config, project="taxi")

# Definir la función de entrenamiento
def train_sweep():
    # inicializar la corrida
    wandb.init()
    
    config = wandb.config
    
    # entrenar el agente
    train(env, Q, config.alpha, config.gamma, config.epsilon, config.epsilon_min, config.epsilon_decay, config.epochs)
    
    # finalizar la corrida
    wandb.finish()

# Ejecutar el agente de sweep
wandb.agent(sweep_id, function=train_sweep)